# Generacion de Datasets utilizando RLHF para fine tunning

## Procedimiento

- Recoleccion de Datos: Obtener Informacion mediante documentos de textos seleccionables (no escaneados), mediante web, redes sociales, scrapping,etc
- Extraer Texto: Usa PyMuPDF para extraer texto de PDFs no escaneados, procesando en paralelo con ProcessPoolExecutor para manejar grandes volúmenes (>100, >10,000).
- Limpiar Datos: Elimina espacios múltiples y caracteres no deseados con expresiones regulares, asegurando texto coherente.
- Validar: Verifica que los fragmentos extraídos no estén vacíos y tengan una longitud mínima (por ejemplo, 50 caracteres).
- Generar Conversaciones: Divide el texto en párrafos, usa un modelo como meta-llama/Llama-3.2-1B-Instruct para generar diálogos conversacionales específicos al contenido, con al menos 4 intercambios por conversación, en formato {"messages": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}], "topic": "..."}.
- Estructurar Dataset: Guarda las conversaciones en archivos JSONL con dos columnas: "messages" y "topic".
Cargar y Subir: Carga el dataset con load_dataset("json", data_files="path/*.jsonl") y súbelo a un repositorio privado en Hugging Face Hub con push_to_hub("username/dataset_name", private=True).

In [2]:
import torch

torch.cuda.is_available()

True

In [128]:
%pip install --upgrade pymupdf nltk ollama


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [176]:
import os
NUM_WORKERS = min(os.cpu_count(), 8)  # Máximo de 8 trabajadores para evitar sobrecarga
BATCH_SIZE = 10  # Fragmentos por trabajador
MAX_LENGTH = 500  # Longitud máxima de la generación
TEMPERATURE = 0.7  # Equilibrio entre creatividad y coherencia
TOP_P = 0.9  # Filtrado de núcleo para diversidad
NUM_RETURN_SEQUENCES = 1  # Una secuencia por fragmento
MIN_FRAGMENT_LENGTH = 50  # Longitud mínima del fragmento
MIN_CONVERSATION_LENGTH = 4  # Mínimo 4 mensajes por conversación
MAX_JSONL_SIZE = 1000  # Máximo de entradas por archivo JSONL

### Recoleccion de Datos

In [177]:
from pathlib import Path

# Verificar si la carpeta existe

folder_url = "/workspace/data/uploads"
folder = Path(folder_url)


if folder.exists() and folder.is_dir():
    print("Valid Folder")
    
# Obtener todos los archivos de la carpeta
files = [f for f in folder.rglob("*") if f.is_file()]

files
len(files)

Valid Folder


94

### Extraccion de Texto y Limpieza de Datos

In [178]:
import pymupdf
import re

output_folder= "extracted"

def clean_text(text):
    """Limpia el texto extraído eliminando espacios múltiples y caracteres no deseados."""
    text = re.sub(r'\s+', ' ', text)  # Reemplaza múltiples espacios por uno solo
    text = re.sub(r'[^\x20-\x7E]', '', text)  # Elimina caracteres no imprimibles
    return text.strip()


NORMALIZE_TEXT = False
def extract_text_from_pdf(pdf_path):
    """Extrae texto de cada página de un PDF y valida que no esté vacío."""
    try:
        doc = pymupdf.open(pdf_path)
        pages_text = []
        for page in doc:
            text = page.get_text()
            cleaned_text =  clean_text(text) if NORMALIZE_TEXT else  text
            if cleaned_text and len(cleaned_text) >= MIN_FRAGMENT_LENGTH:
                pages_text.append(cleaned_text)
        doc.close()
        return pages_text if pages_text else None
    except Exception as e:
        print(f"Error procesando {pdf_path}: {e}")
        return None
    

In [179]:
chunks = extract_text_from_pdf(files[0])

chunks[0]

'Comité de Supervisión\nBancaria de Basilea\nDocumento Consultivo\nVisión General del Nuevo\nAcuerdo de Capital de\nBasilea\nEmitido para consulta hasta el 31 de mayo de 2001\nTraducción realizada por la Asociación de Supervisores Bancarios de\nlas Américas (ASBA). En caso de dudas consultar el texto original en\ninglés\nEnero 2001\n'

### Generar Conversaciones

In [180]:

import nltk
# Descarga recursos de NLTK para segmentación
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [181]:
from nltk import sent_tokenize
import ollama
from pydantic import BaseModel, Field
from typing import List
OLLAMA_MODEL="llama3.1:8b"

# Definir el esquema Pydantic para la salida estructurada
class Message(BaseModel):
    role: str = Field(..., pattern="^(user|assistant)$")
    content: str

class Conversation(BaseModel):
    messages: List[Message] = Field(..., min_items=MIN_CONVERSATION_LENGTH)
    topic: str


def generate_conversation(fragment):
    """Genera una conversación estructurada en formato JSON usando Ollama con el parámetro format."""
    if not fragment or len(fragment) < MIN_FRAGMENT_LENGTH:
        return None

    # Divide el fragmento en oraciones
    sentences = sent_tokenize(fragment)
    if not sentences:
        return None

    # Usa la primera oración como tema inicial
    topic = sentences[0][:50] + "..." if len(sentences[0]) > 50 else sentences[0]

    # Prompt para guiar la generación
    prompt = f"""
    Basándote únicamente en el siguiente texto:
    
    '{fragment[:500]}'
    
    Crea una conversación entre un usuario y un asistente con el siguiente comportamiento:
    
    1. La conversación debe tener entre 2 y 8 intercambios, dependiendo de la cantidad de contexto útil disponible en el texto. Si el texto tiene poco contexto, limita la conversación a solo 1 intercambio (usuario + asistente). Si es más informativo, genera una conversación de entre 4 a 8 mensajes.
    2. Las intervenciones del usuario pueden incluir:
        - Preguntas específicas y relevantes sobre el contenido (no genéricas como “¿Cuál es el tema del texto?” ni “¿Qué organización se menciona?”).
        - Frases incompletas o afirmaciones a medio terminar para que el asistente las complete, como: “El fenómeno descrito es...” o “La causa principal de esto es...”.
    3. Las respuestas del asistente deben ser claras, coherentes y basarse únicamente en el contenido del texto.
    4. El hilo de la conversación debe seguir una lógica progresiva: cada mensaje debe conectar naturalmente con el anterior.
    5. Identifica el **tema principal** del texto y agrégalo como valor del campo `topic`.
    
    Este resultado está orientado a la **generación de datasets para aprendizaje por refuerzo** (Reinforcement Learning) o aprendizaje supervisado. Por tanto:
    - La conversación debe reflejar interacciones verosímiles y útiles entre humanos y asistentes conversacionales.
    - Debe permitir entrenar modelos que comprendan el contexto, anticipen respuestas coherentes y mantengan un flujo natural.
    
    Ejemplo de interacción válida:
    
    user: "El experimento de dispersión muestra que..."
    
    assistant: "…la luz blanca se divide en varios colores debido a su paso por un prisma."
    
    Otro ejemplo:
    
    user: "¿Por qué se utilizan prismas en el experimento?"
    
    assistant: "Porque permiten observar cómo se separan los diferentes componentes del espectro de luz blanca."
    
    Asegúrate de que las preguntas no sean genéricas, que la conversación tenga coherencia, y que se respete el formato.
    """

    try:
        # Llama a Ollama con el esquema Pydantic
        response = ollama.chat(
            model=OLLAMA_MODEL,
            messages=[{"role": "user", "content": prompt}],
            options={
                "temperature": TEMPERATURE,
                "top_p": TOP_P,
            },
            format=Conversation.model_json_schema()  # Especifica el esquema JSON
        )
        return Conversation.model_validate_json(response.message.content)
    except Exception as e:
        print(f"Error generando conversación con Ollama: {e}")
        # Fallback en caso de error
        return None

In [182]:
import json
def process_pdf(index, pdf_path, output_dir):
    """Procesa un PDF, genera conversaciones y las guarda en JSONL."""
    pages_text = extract_text_from_pdf(pdf_path)
    if not pages_text:
        return

    jsonl_file = os.path.join(output_dir, f"pdf_{index:04d}.jsonl")
    with open(jsonl_file, "w", encoding="utf-8") as f:
        for page_text in pages_text:
            # Divide el texto de la página en fragmentos (párrafos)
            fragments = page_text.split('\n\n')  # Asume que los párrafos están separados por líneas en blanco
            for fragment in fragments:
                if len(fragment) > 20:  # Ignora fragmentos cortos
                    conversation = generate_conversation(fragment)
                    if conversation:
                        f.write(json.dumps(conversation.model_dump(), ensure_ascii=False) + "\n")

In [183]:
len(files)

94

In [ ]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm



pdf_files = files[:5]
output_dir="outputs"

num_workers = int(min(multiprocessing.cpu_count() * 0.8,len(pdf_files)))

def process_pdf_wrapper(args):
    index, pdf_path, output_dir = args
    process_pdf(index, pdf_path, output_dir)


with ProcessPoolExecutor(max_workers=num_workers) as executor:
    list(tqdm(executor.map(process_pdf_wrapper, [(i, p, output_dir) for i, p in enumerate(pdf_files)]), total=len(pdf_files)))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

### Saving

In [190]:
from datasets import load_dataset


dataset = load_dataset("json", data_files="./outputs/*.jsonl")

dataset


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages', 'topic'],
        num_rows: 225
    })
})

In [191]:
from huggingface_hub import login
login()

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########| 71.7kB / 71.7kB            

CommitInfo(commit_url='https://huggingface.co/datasets/jeanmcm/b_risks/commit/a534f757b5961e3c0f354726c09c8b960018bd7e', commit_message='Upload dataset', commit_description='', oid='a534f757b5961e3c0f354726c09c8b960018bd7e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jeanmcm/b_risks', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jeanmcm/b_risks'), pr_revision=None, pr_num=None)

In [ ]:
dataset.push_to_hub("jeanmcm/b_risks", private=True)

# Testing RAG with Open WebUI 

In [36]:
import requests
import json
import sys
import time
from IPython.display import display, clear_output, HTML

url = "https://vwlppjjfa98c9x-8080.proxy.runpod.net"
api_key ="sk-05568562f28844fe997cadf960a346cd"

messages =  [{"role": "user", "content": "Que es el Riesgo Financiero?"}]

try:
    # Realizar la solicitud con stream=True
    with requests.post(f"{url}/api/chat/completions", stream=True,headers={
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    },json={
      "model":"bosft-riesgos-rag-model",
      "messages":messages,
      "stream":True
      }) as response:
        response.raise_for_status()
        # Variable para almacenar la salida acumulada
        accumulated_output = ""

        # Iterar sobre las líneas de la respuesta
        for line in response.iter_lines():
            if line:
                # Decodificar la línea
                decoded_line = line.decode('utf-8').strip()
                # Si la línea comienza con "data:", extraer el contenido
                if decoded_line.startswith("data:"):
                    decoded_line = decoded_line[5:].strip()  # Quitar "data: "

                # Ignorar líneas vacías o marcadores de fin como "[DONE]"
                if not decoded_line or decoded_line == "[DONE]" :
                    continue
                
                
                try:
                    # Parsear si es JSON
                    data = json.loads(decoded_line)
                    if "choices" not in data: continue
                    
                    delta = data['choices'][0]['delta']
                    if "content" in delta: new_data = delta['content']
                except json.JSONDecodeError:
                    # Si no es JSON, usar la línea como texto
                    new_data = decoded_line

                # Acumular y mostrar la salida dinámicamente
                if new_data:
                    accumulated_output += new_data
                    # Limpiar la salida anterior y mostrar la nueva
                    clear_output(wait=True)
                    display(HTML(f"<b>Respuesta en streaming:</b> {accumulated_output}"))
                    time.sleep(0.1)  # Pequeña pausa para visibilidad
                    

except requests.exceptions.RequestException as e:
    print(f"\nError en la solicitud: {e}")

# Testing with Flowise

In [ ]:
# todo